In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaForSequenceClassification
from transformers import AdamW
import random
import matplotlib.pyplot as plt
from collections import OrderedDict
from scipy.spatial.distance import cosine
from sim_utils import load_examples, Inputexample, CustomTextDataset, freeze_layers, train, test
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig, AutoModel, AutoTokenizer

os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [2]:
N = 10
data = []
labels = []

train_samples = []
train_labels = []

valid_samples = []
valid_labels = []

test_samples = []
test_labels = []

embed_dim = 768
batch_size = 16 
lr=2e-3  # you can adjust 
temp = 0.3  # you can adjust 
lamda = 0.01  # you can adjust  
skip_time = 0 # the number of time that yi not equal to yj in supervised contrastive loss equation 

data_names = ['BANKING77','HWU64'] #'CLINC150',  
model_names = ['sup-simcse-roberta-base','unsup-simcse-roberta-base','sup-simcse-bert-base-uncased','unsup-simcse-bert-base-uncased']
shot_names = ['train_5','train_10']
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [3]:
!nvidia-smi

Sun May  1 15:01:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:84:00.0 Off |                  N/A |
| 24%   28C    P8    10W / 250W |      3MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:85:00.0 Off |                  N/A |
| 22%   

In [4]:
for shot_name in shot_names:
    for data_name in data_names:

        path_shot = f'../../../{data_name}/{shot_name}/'

        valid_path = f'../../../{data_name}/valid/'
        test_path = f'../../../{data_name}/test/'


        # load data
        train_samples = load_examples(path_shot)
        valid_samples = load_examples(valid_path)
        test_samples = load_examples(test_path)


        print("===== small train set ====")
        
        
        data = []
        labels = []

        for i in range(len(train_samples)):
            data.append(train_samples[i].text)
            labels.append(train_samples[i].label)


        train_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=False)
        train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)



        print("===== validation set ====")

        data = []
        labels = []

        for i in range(len(valid_samples)):
            data.append(valid_samples[i].text)
            labels.append(valid_samples[i].label)

        valid_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=False)
        valid_loader = DataLoader(valid_data,batch_size=batch_size,shuffle=True)



        print("===== test set ====")

        data = []
        labels = []

        for i in range(len(test_samples)):
            data.append(test_samples[i].text)
            labels.append(test_samples[i].label)

        test_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=False)
        test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=True)



         # got the number of unique classes from dataset
        num_class = len(np.unique(np.array(labels)))

         # get text label of uniqure classes
        unique_label = np.unique(np.array(labels))

         # map text label to index classes
        label_maps = {unique_label[i]: i for i in range(len(unique_label))}

        print("label_maps :",label_maps)
        print("num_class:",num_class)


        lines = ['test acc']
        for model_name in model_names:



            exp_name = f'{model_name}_lr={lr}_t={temp}_{data_name}_{shot_name}'
            direct_name = f"princeton-nlp/{model_name}"

            print("direct_name :",direct_name)
            tokenizer = AutoTokenizer.from_pretrained(direct_name)
            config = AutoConfig.from_pretrained(direct_name)
            config.num_labels = num_class
            simcse = AutoModelForSequenceClassification.from_pretrained(direct_name,config=config)

            simcse = freeze_layers(simcse,freeze_layers_count=12)
            optimizer= AdamW(simcse.parameters(), lr=lr)
            simcse = simcse.to(device)

            train_log, valid_log = train(exp_name,simcse,device,label_maps,optimizer,train_loader,valid_loader,train_data,valid_data,tokenizer,epochs=30)




            PATH = f'../../../fewshot_models/{exp_name}.pth'
            best_model = AutoModelForSequenceClassification.from_pretrained(direct_name,config=config)
            # Model class must be defined somewhere
            best_model.load_state_dict(torch.load(PATH))
            best_model = best_model.to(device)


            test_acc = test(best_model,device,label_maps,test_loader,len(test_data),tokenizer)

            test_acc = 100 * test_acc
            res = f'data_name:{data_name}_model:{model_name}_test_acc:{str(test_acc)}'
            lines.append(res)


        with open(f'result_{data_name}_{shot_name}.txt', 'w') as f:
            for line in lines:
                f.write(line)
                f.write('\n')

===== small train set ====
Train on Cross Entropy loss
len of dataset : 385
===== validation set ====
Train on Cross Entropy loss
len of dataset : 1540
===== test set ====
Train on Cross Entropy loss
len of dataset : 3080
label_maps : {'Refund_not_showing_up': 0, 'activate_my_card': 1, 'age_limit': 2, 'apple_pay_or_google_pay': 3, 'atm_support': 4, 'automatic_top_up': 5, 'balance_not_updated_after_bank_transfer': 6, 'balance_not_updated_after_cheque_or_cash_deposit': 7, 'beneficiary_not_allowed': 8, 'cancel_transfer': 9, 'card_about_to_expire': 10, 'card_acceptance': 11, 'card_arrival': 12, 'card_delivery_estimate': 13, 'card_linking': 14, 'card_not_working': 15, 'card_payment_fee_charged': 16, 'card_payment_not_recognised': 17, 'card_payment_wrong_exchange_rate': 18, 'card_swallowed': 19, 'cash_withdrawal_charge': 20, 'cash_withdrawal_not_recognised': 21, 'change_pin': 22, 'compromised_card': 23, 'contactless_not_working': 24, 'country_support': 25, 'declined_card_payment': 26, 'decli

Some weights of the model checkpoint at princeton-nlp/sup-simcse-roberta-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model 

classifier.dense.weight
classifier.dense.bias
classifier.out_proj.weight
classifier.out_proj.bias
======  Epoch 1 ====== 
 Training Loss: 0.2769007527983034, 		 Training acc: 0.06493506580591202
train correct :  tensor(25, device='cuda:1')
train total : 385
 Validation Loss: 0.18952957422702343, 		 Validation acc: 0.3603895902633667
valid correct :  tensor(555, device='cuda:1')
valid total : 1540
Validation Loss Decreased(inf--->0.189530) 	 Saving The Model
======  Epoch 2 ====== 
 Training Loss: 0.1514880796531578, 		 Training acc: 0.4623376429080963
train correct :  tensor(178, device='cuda:1')
train total : 385
 Validation Loss: 0.12453962053571428, 		 Validation acc: 0.5077921748161316
valid correct :  tensor(782, device='cuda:1')
valid total : 1540
Validation Loss Decreased(0.189530--->0.124540) 	 Saving The Model
======  Epoch 3 ====== 
 Training Loss: 0.0842471457146979, 		 Training acc: 0.7116882801055908
train correct :  tensor(274, device='cuda:1')
train total : 385
 Validati

Some weights of the model checkpoint at princeton-nlp/sup-simcse-roberta-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model 

correct : 2255
total : 3080
direct_name : princeton-nlp/unsup-simcse-roberta-base


...

correct : 801
total : 1076
direct_name : princeton-nlp/unsup-simcse-bert-base-uncased


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/unsup-simcse-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


classifier.weight
classifier.bias
======  Epoch 1 ====== 
 Training Loss: 0.2516597412526608, 		 Training acc: 0.11562500149011612
train correct :  tensor(37, device='cuda:1')
train total : 320
 Validation Loss: 0.22582645265586315, 		 Validation acc: 0.35501858592033386
valid correct :  tensor(382, device='cuda:1')
valid total : 1076
Validation Loss Decreased(inf--->0.225826) 	 Saving The Model
======  Epoch 2 ====== 
 Training Loss: 0.19141902700066565, 		 Training acc: 0.690625011920929
train correct :  tensor(221, device='cuda:1')
train total : 320
 Validation Loss: 0.18684529460495733, 		 Validation acc: 0.5613383054733276
valid correct :  tensor(604, device='cuda:1')
valid total : 1076
Validation Loss Decreased(0.225826--->0.186845) 	 Saving The Model
======  Epoch 3 ====== 
 Training Loss: 0.14383087456226348, 		 Training acc: 0.8687500357627869
train correct :  tensor(278, device='cuda:1')
train total : 320
 Validation Loss: 0.15632289382161704, 		 Validation acc: 0.61710035800

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/unsup-simcse-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


correct : 727
total : 1076


In [5]:
label_maps

{'alarm_query': 0,
 'alarm_remove': 1,
 'alarm_set': 2,
 'audio_volume_down': 3,
 'audio_volume_mute': 4,
 'audio_volume_up': 5,
 'calendar_query': 6,
 'calendar_remove': 7,
 'calendar_set': 8,
 'cooking_recipe': 9,
 'datetime_convert': 10,
 'datetime_query': 11,
 'email_addcontact': 12,
 'email_query': 13,
 'email_querycontact': 14,
 'email_sendemail': 15,
 'general_affirm': 16,
 'general_commandstop': 17,
 'general_confirm': 18,
 'general_dontcare': 19,
 'general_explain': 20,
 'general_joke': 21,
 'general_negate': 22,
 'general_praise': 23,
 'general_quirky': 24,
 'general_repeat': 25,
 'iot_cleaning': 26,
 'iot_coffee': 27,
 'iot_hue_lightchange': 28,
 'iot_hue_lightdim': 29,
 'iot_hue_lightoff': 30,
 'iot_hue_lighton': 31,
 'iot_hue_lightup': 32,
 'iot_wemo_off': 33,
 'iot_wemo_on': 34,
 'lists_createoradd': 35,
 'lists_query': 36,
 'lists_remove': 37,
 'music_likeness': 38,
 'music_query': 39,
 'music_settings': 40,
 'news_query': 41,
 'play_audiobook': 42,
 'play_game': 43,
 'p

In [6]:
lines

['test acc',
 'data_name:HWU64_model:sup-simcse-roberta-base_test_acc:75.0',
 'data_name:HWU64_model:unsup-simcse-roberta-base_test_acc:65.33457249070632',
 'data_name:HWU64_model:sup-simcse-bert-base-uncased_test_acc:74.44237918215613',
 'data_name:HWU64_model:unsup-simcse-bert-base-uncased_test_acc:67.56505576208178']

In [7]:
exp_name

'unsup-simcse-bert-base-uncased_lr=0.002_t=0.3_HWU64_train_5'